### ALERT GENERATION

In [1]:
# !pip install transformers sentencepiece torch
# !pip install gensim
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BartForConditionalGeneration, GPT2Tokenizer, GPT2LMHeadModel, AutoProcessor, LlavaForConditionalGeneration
from typing import Tuple, List, Dict, Any

import pandas as pd
import random
import re
import os
import matplotlib.pyplot as plt

from gensim.models.keyedvectors import load_word2vec_format

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.utils.tensorboard import SummaryWriter
from torch.jit import RecursiveScriptModule

SEED = 2222

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

import gzip
import shutil
import json
import accelerate

2025-04-20 16:18:23.389427: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-20 16:18:23.399822: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745165903.413492    7357 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745165903.417436    7357 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745165903.428244    7357 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

### BART-LARGE-CNN

In [2]:
# BART
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

input_data = {
    "text": "Financial analysts point out that earnings could impact ExxonMobil significantly",
    "ner": [('ExxonMobil', 'ORG')],
    "sentiment": "positive"
}
text = input_data["text"]
ner = input_data["ner"]
sentiment = input_data["sentiment"]

# NER in legible format
entity_text = ", ".join([f"{ent[0]} ({ent[1]})" for ent in ner])

# Input text
prompt = (
        f"As an experienced investment advisor, provide financial advice based on the following analysis: "
        f"'{text}', which has a {sentiment} sentiment towards the company/entity '{entity_text}'. "
        f"Advise the investor on how to proceed with their investment in this company, considering the current situation and sentiment."
    )

inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=30, length_penalty=2.0, num_beams=4)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("\nResumen generado:\n", summary)


Resumen generado:
 As an experienced investment advisor, provide financial advice based on the following analysis. Advise the investor on how to proceed with their investment in this company, considering the current situation and sentiment.


### T5-SMALL

In [3]:
# T5-small
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    inputs["input_ids"],
    max_new_tokens=60,
    num_beams=4,
    no_repeat_ngram_size=2,
    early_stopping=True
)

summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Resumen generado:", summary)

Resumen generado: the investor on how to proceed with their investment in this company, considering the current situation and sentiment.


### GPT-2

In [4]:
# GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

input_data = {
    "text": "Financial analysts point out that earnings could impact ExxonMobil significantly",
    "ner": [('ExxonMobil', 'ORG')],
    "sentiment": "positive"
}

text = input_data["text"]
ner = input_data["ner"]
sentiment = input_data["sentiment"]

# Legible NER
entity_text = ", ".join([f"{ent[0]} ({ent[1]})" for ent in ner])

prompt = (
        f"As an experienced investment advisor, provide financial advice based on the following analysis: "
        f"'{text}', which has a {sentiment} sentiment towards the company/entity '{entity_text}'. "
        f"Advise the investor on how to proceed with their investment in this company, considering the current situation and sentiment."
    )

inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = model.generate(
    inputs["input_ids"],
    max_length=100,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("RESUMEN:" , generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


RESUMEN: As an experienced investment advisor, provide financial advice based on the following analysis: 'Financial analysts point out that earnings could impact ExxonMobil significantly', which has a positive sentiment towards the company/entity 'ExxonMobil (ORG)'. Advise the investor on how to proceed with their investment in this company, considering the current situation and sentiment.

How to Invest in ExxonMobil

ExxonMobil is an established company, and has been around for a long time. Its reputation has been


### FLAN-T5

In [5]:
# FLAN-T5
model_name = "google/flan-t5-large" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate_investment_advice_flan(input_data):
    text = input_data["text"]
    ner = input_data["ner"]
    sentiment = input_data["sentiment"]
    caption = input_data["caption"]

    entity_text = ", ".join([f"{ent[0]} ({ent[1]})" for ent in ner])

    prompt = (
        f"Based on the following information, provide investment advice:\n\n"
        f"Text analysis: '{text}'\n"
        f"Sentiment: {sentiment}\n"
        f"Entity mentioned: '{entity_text}'\n"
        f"Image caption: '{caption}'\n\n"
        f"Given this context (take into account positive leads to invest and negative not to invest), what should an investor do regarding this company?"
    )


    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

    outputs = model.generate(
        **inputs,
        max_length=128,
        temperature=0.7,
        num_beams=4,
        early_stopping=True
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

# Input
input_data = {
    "text": "Financial analysts point out that earnings could impact ExxonMobil significantly",
    "ner": [('ExxonMobil', 'ORG')],
    "sentiment": "positive",
    "caption": None
}

# Advice
advice = generate_investment_advice_flan(input_data)
print("Generated Advice:\n", advice)

/opt/conda/envs/kernel_personalizado/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated Advice:
 Invest


### SA WITH ALERT

In [6]:
with gzip.open("GoogleNews-vectors-negative300.bin.gz", "rb") as f_in:
    with open("GoogleNews-vectors-negative300.bin", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [7]:
w2v_model = load_word2vec_format("GoogleNews-vectors-negative300.bin", binary = True)

In [8]:
def load_data_ner(file_path: str) -> Tuple[List[List[str]], List[int]]:
    """
    Load data from a specified file path, extract texts and targets, and tokenize the texts using the tokenize_tweet function.

    Parameters:
    file_path (str): The path to the dataset file.

    Returns:
    Tuple[List[str], List[int]]: Lists of texts and corresponding targets.
    """
    df = pd.read_csv(file_path)
    
    sentences = []
    sentence_labels = []
    current_sentence = []
    current_labels = []
    current_sent_idx = None

    for _, row in df.iterrows():
        word = row['gold_token']
        label = row['gold_label']
        sent_idx = row['sent_idx']

        if sent_idx != current_sent_idx:
            if current_sentence:
                sentences.append(current_sentence)
                sentence_labels.append(current_labels)
            current_sentence = []
            current_labels = []
            current_sent_idx = sent_idx

        current_sentence.append(word)
        current_labels.append(label)

    # The last sentence if it is not empty
    if current_sentence:
        sentences.append(current_sentence)
        sentence_labels.append(current_labels)

    return sentences, sentence_labels

In [9]:
tr_texts, tr_targets = load_data_ner('finer_ord_train.csv')
vl_texts, vl_targets = load_data_ner('finer_ord_validation.csv')
ts_texts, ts_targets = load_data_ner('finer_ord_test.csv')

training_data = pd.DataFrame({'Tweets': tr_texts, 'Labels': tr_targets})
training_data[:4]

,Tweets,Labels
0,"[Kenyan, Firms, Eye, Deals, During, Obama, Sum...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[By, Neville, Otuki, Kenya, 's, business, lead...","[0, 1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Industrialists, ,, entrepreneurs, and, banker...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[More, on, This, Kenya, :, Mombasa, Road, ,, U...","[0, 0, 0, 3, 0, 3, 4, 0, 3, 4, 0, 3, 4, 4, 4, ..."


In [10]:
class NERDataset(Dataset):
    """
    A PyTorch Dataset for the dataset.
    

    Attributes:
        texts (List[List[str]]): List of se tokens.
        targets (List[List[int]]): List of target labels.
    """

    def __init__(self,
                 texts: List[List[str]],
                 targets: List[List[int]]
                 ):
        """
        Initializes the NERDataset with the given file path.

        Args:
            texts (List[List[str]]): List of tweets tokens.
            targets (List[List[int]]): List of target labels.
        """
        self.texts = texts
        self.targets = targets

    def __len__(self) -> int:
        """Returns the length of the dataset."""
        return len(self.targets)

    def __getitem__(self, idx: int) -> Tuple[List[List[str]], List[int]]:
        """
        Returns the embedded tensor and target for the text at the specified index.

        Args:
            idx (int): Index of the item.

        Returns:
            Tuple[List[str], List[int]]: A tuple containing the texts and the target label for idx.
        """
        return self.texts[idx], self.targets[idx]

In [11]:
tr_dataset = NERDataset(tr_texts, tr_targets)
vl_dataset = NERDataset(vl_texts, vl_targets)
ts_dataset = NERDataset(ts_texts, ts_targets)

In [12]:
def word2idx(embedding_model: Any, sentence: List[str], labels:List[int]) -> torch.Tensor:
    """
    Converts a sentence to a list of word indices based on an embedding model.

    This function iterates through each word in the sentence and retrieves its corresponding index
    from the embedding model's vocabulary. If a word is not present in the model's vocabulary,
    it is skipped.

    Args:
        embedding_model (Any): The embedding model with a 'key_to_index' attribute, which maps words to their indices.
        sentence (List[str]): A list of words representing the sentence.

    Returns:
        torch.Tensor: A tensor of word indices corresponding to the words in the tweet.
    """
    index = []
    labels_out = [] # We should eliminate labels from the words that do not exist in w2v
    for word, label in zip(sentence, labels):
        if word in embedding_model.key_to_index:
            index.append(embedding_model.key_to_index.get(word))
            labels_out.append(label)
    
    return torch.tensor(index, dtype=torch.long), torch.tensor(labels_out, dtype=torch.long)


def calculate_accuracy(model: torch.nn.Module, dataloader: DataLoader, threshold: float = 0.5, device: str = 'cpu') -> float:
    """
    Calculate the accuracy of a PyTorch model given a DataLoader.

    The function moves the model to the specified device, sets it to evaluation mode, and computes
    the accuracy by comparing the model's predictions against the true labels. The predictions are
    determined based on a specified threshold.

    Args:
        model (torch.nn.Module): The PyTorch model to evaluate.
        dataloader (DataLoader): The DataLoader containing the dataset to evaluate against.
        threshold (float, optional): Probability threshold to predict a sample as positive. Defaults to 0.5.
        device (str, optional): Device to which the model and data are moved ('cpu' or 'cuda'). Defaults to 'cpu'.

    Returns:
        float: The accuracy of the model on the given dataset.
    """
    model.to(device)
    model.eval()
    correct_predictions = 0
    total_tokens = 0

    with torch.no_grad():
        for features, labels, lenghts in dataloader:
            features = features.to(device)
            labels = labels.to(device)
            output = model(features, torch.Tensor(lenghts))

            predictions = torch.argmax(torch.softmax(output, dim=2), dim=2)

            # mask for valid tokens (label != -1)
            mask = labels != -1

            correct_predictions += (predictions[mask] == labels[mask]).sum().item()
            total_tokens += mask.sum().item()

    accuracy = correct_predictions / total_tokens if total_tokens > 0 else 0.0
    return accuracy


def collate_fn(batch: List[Tuple[List[str], int]]) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Prepares and returns a batch for training/testing in a torch model.

    This function sorts the batch by the length of the text sequences in descending order,
    tokenizes the text using a pre-defined word-to-index mapping, pads the sequences to have
    uniform length, and converts labels to tensor.

    Args:
        batch (List[Tuple[List[str], int]]): A list of tuples, where each tuple contains a
                                             list of words (representing a text) and an integer label.

    Returns:
        Tuple[torch.Tensor, torch.Tensor, torch.Tensor]: A tuple containing three elements:
            - texts_padded (torch.Tensor): A tensor of padded word indices of the text.
            - labels (torch.Tensor): A tensor of labels.
            - lengths (torch.Tensor): A tensor representing the lengths of each text sequence.
    """
    # Sort the batch by the length of text sequences in descending order
    sorted_batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)

    # Unzip texts and labels from the sorted batch
    texts, labels = [text for text, label in sorted_batch], [label for text, label in sorted_batch]

    # Convert texts to indices using the word2idx function and w2v_model
    texts_indx: List[torch.Tensor] = []
    labels_indx: List[torch.Tensor] = []
    for sentence, label in zip(texts, labels):
        text_indx, label_indx = word2idx(w2v_model, sentence, label)
        texts_indx.append(text_indx)
        labels_indx.append(label_indx)

    # Calculate the lengths of each element of texts_indx.
    # The minimum length shall be 1, in order to avoid later problems when training the RNN
    lengths: List[torch.Tensor] = [torch.tensor(len(text_indx)) for text_indx in texts_indx]
    
    # Pad the text sequences to have uniform length
    texts_padded: torch.Tensor = torch.nn.utils.rnn.pad_sequence(texts_indx, batch_first=True)
    labels_padded: torch.Tensor = torch.nn.utils.rnn.pad_sequence(labels_indx, batch_first=True, padding_value=-1)

    return texts_padded, labels_padded, lengths


def collate_fn_pre_padding(batch: List[Tuple[List[str], int]]) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    sorted_batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)

    texts, labels = [text for text, label in sorted_batch], [label for text, label in sorted_batch]

    texts_indx: List[torch.Tensor] = []
    labels_indx: List[torch.Tensor] = []
    for sentence, label in zip(texts, labels):
        text_indx, label_indx = word2idx(w2v_model, sentence, label)
        texts_indx.append(text_indx)
        labels_indx.append(label_indx)

    max_len = max(len(t) for t in texts_indx)

    texts_padded = []
    labels_padded = []
    lengths = []

    for text, label in zip(texts_indx, labels_indx):
        pad_len = max_len - len(text)
        texts_padded.append(F.pad(text, (pad_len, 0), value=0))        
        labels_padded.append(F.pad(label, (pad_len, 0), value=-1))    
        lengths.append(torch.tensor(len(text)))

    return torch.stack(texts_padded), torch.stack(labels_padded), torch.stack(lengths)


torch.cuda.empty_cache()
embedding_weights = torch.Tensor(w2v_model.vectors).to("cpu")

# Define configuration for MLP Classifier training
batch_size: int = 64
epochs: int = 100
print_every: int = 5
patience: int = 5
learning_rate: float = 0.001
hidden_dims: List[int] = 64
num_layers: int = 3

# Check if GPU is available and move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_torch_model(model, train_dataloader, val_dataloader, criterion, optimizer, epochs, print_every, patience, device):
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    train_accuracies: Dict[int, float] = {}
    val_accuracies: Dict[int, float] = {}

    model.to(device) 

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        torch.cuda.empty_cache()
        for i, (x_batch, y_batch, lengths) in enumerate(train_dataloader):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            lengths = torch.Tensor(lengths).to(device) 

            optimizer.zero_grad()

            # Forward pass
            logits = model(x_batch, lengths)

            logits = logits.view(-1, logits.shape[-1])
            y_batch = y_batch.view(-1).long()

            loss = criterion(logits, y_batch)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x_batch, y_batch, lengths in val_dataloader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                lengths = torch.Tensor(lengths).to(device)  

                logits = model(x_batch, lengths)

                logits = logits.view(-1, logits.shape[-1])
                y_batch = y_batch.view(-1).long()

                loss = criterion(logits, y_batch)
                val_loss += loss.item()

        if epoch % print_every == 0 or epoch == epochs - 1:
            acc_t = calculate_accuracy(model, train_dataloader, device=device)
            acc_v = calculate_accuracy(model, val_dataloader, device=device)

            train_accuracies[epoch] = acc_t
            val_accuracies[epoch] = acc_v

            avg_train_loss = train_loss / len(train_dataloader)
            avg_val_loss = val_loss / len(val_dataloader)

            print(f"Epoch {epoch+1}/{epochs}: ")
            print(f"Accuracy: train {acc_t} - test {acc_v}")
            print(f"Loss: train {avg_train_loss} - test: {avg_val_loss}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f'Early stopping triggered at epoch {epoch + 1}')
            break

    return train_accuracies, val_accuracies

In [13]:
def load_data(json_path: str) -> List[Tuple[str, List[Tuple[str, str]]]]:
    """
    Carga datos con estructura [texto, {entities: [[start, end, label], ...]}]

    Devuelve: lista de tuplas (texto, entidades extraídas como (entidad, tipo))
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    data = []

    for item in raw_data:
        if not isinstance(item, list) or len(item) != 2:
            continue  # Por si hay errores de formato

        text = item[0]
        entity_info = item[1]
        entities = entity_info.get("entities", [])

        extracted_entities = [(text[start:end], label) for start, end, label in entities]

        data.append((text, extracted_entities))

    return data

In [14]:
test_data = load_data("finer_test_data.json")

In [15]:
def word2idx(embedding_model: Any, text: List[str]) -> torch.Tensor:
    indexes = []

    for word in text:
      if word in embedding_model.key_to_index:
        indexes.append(embedding_model.key_to_index[word])

    return indexes

def preprocess_single_text(text: str, w2v_model, word2idx_fn):
    idxs = word2idx_fn(w2v_model, text)
    text_tensor = torch.tensor(idxs, dtype=torch.long).unsqueeze(0)
    length = torch.tensor([len(idxs)])
    return text_tensor, length

In [16]:
class TCNBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation):
        super().__init__()
        self.conv = torch.nn.Conv1d(
            in_channels, 
            out_channels, 
            kernel_size=kernel_size, 
            padding=(kernel_size - 1) * dilation, 
            dilation=dilation
        )
        self.relu = torch.nn.ReLU()
        self.bn = torch.nn.BatchNorm1d(out_channels)
        self.dropout = torch.nn.Dropout(0.2)

    def forward(self, x):
        out = self.conv(x)
        out = out[:, :, :-self.conv.padding[0]] 
        out = self.relu(out)
        out = self.bn(out)
        return self.dropout(out)

class SentimentTCN(torch.nn.Module):
    def __init__(self, embedding_weights: torch.Tensor, num_classes: int = 2):
        super().__init__()
        embedding_dim: int = embedding_weights.shape[1]
        self.embedding: torch.nn.Embedding = torch.nn.Embedding.from_pretrained(embedding_weights, freeze=True)

        self.tcn1 = TCNBlock(embedding_dim, 128, kernel_size=3, dilation=1)
        self.tcn2 = TCNBlock(128, 128, kernel_size=3, dilation=2)
        self.tcn3 = TCNBlock(128, 128, kernel_size=3, dilation=4)

        self.pool = torch.nn.AdaptiveMaxPool1d(1)
        self.fc = torch.nn.Linear(128, num_classes)

    def forward(self, x: torch.Tensor, text_lengths: torch.Tensor) -> torch.Tensor:
        embedded: torch.Tensor = self.embedding(x)          
        embedded = embedded.transpose(1, 2)                 

        x = self.tcn1(embedded)
        x = self.tcn2(x)
        x = self.tcn3(x)

        x = self.pool(x).squeeze(2)                       
        output = self.fc(x)                               
        return output


In [17]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

embedding_weights = torch.tensor(w2v_model.vectors)
sa_model = SentimentTCN(embedding_weights)

sa_model.load_state_dict(torch.load("sa_model.pt"))

sa_model.eval()
sa_model.to(DEVICE)

SentimentTCN(
  (embedding): Embedding(3000000, 300)
  (tcn1): TCNBlock(
    (conv): Conv1d(300, 128, kernel_size=(3,), stride=(1,), padding=(2,))
    (relu): ReLU()
    (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (tcn2): TCNBlock(
    (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
    (relu): ReLU()
    (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (tcn3): TCNBlock(
    (conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(4,))
    (relu): ReLU()
    (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (pool): AdaptiveMaxPool1d(output_size=1)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

In [18]:
ner = torch.load("rnn_model_hd32_nl5_batch65_epochs100_dropout0_bidirectional.pt", weights_only=False)
ner.to(DEVICE)
dicc_ner = { 0: "O", 1: "PER_B" ,  2: "PER_I" , 3: "LOC_B", 4: "LOC_I", 5: "ORG_B", 6: "ORG_I"}

/opt/conda/envs/kernel_personalizado/lib/python3.10/site-packages/torch/serialization.py:1434: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


In [19]:
def generate_alert_image(image):
    model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf", torch_dtype=torch.float16, device_map="auto")
    processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
    
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "image", "url": image},
                {"type": "text", "text": "Make a quick and direct conclusion from this graphic."},
            ],
        },
    ]
    
    inputs = processor.apply_chat_template(
        conversation,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to(model.device, torch.float16)
    
    # Generate
    generate_ids = model.generate(**inputs, max_new_tokens=50)
    output = processor.batch_decode(generate_ids, skip_special_tokens=True)
    
    # Extract string from the list
    text = output[0]
    description = text.split("ASSISTANT:")[1].strip()
    return description

### GENERATE ALERT FROM A SINGLE TEXT AND IMAGE

In [ ]:
# Examples
text = "Amanda Butler, a well-known Director, mentions that losses might influence Burberry's recover."
url = "https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F43be4e18-6efd-472e-b6eb-fa27f933015b_3600x3600.png"
# text = "Bitcoin's potential growth: experts suggest positive outlook despite market fluctuations"
# url = "https://www.economist.com/sites/default/files/images/2021/01/articles/main/20210109_woc295.png"

y_pred = []
text_tensor, length = preprocess_single_text(text, w2v_model, word2idx)

text_tensor = text_tensor.to(DEVICE)
length = length.to(DEVICE)

ner_output = ner(text_tensor, torch.tensor([len(text.split(" "))]))

predicted_classes = torch.argmax(ner_output.squeeze(0), dim=1)

words = text.split(" ")
ner_result = [(word, dicc_ner[label.item()]) for word, label in zip(words, predicted_classes) if dicc_ner[label.item()] != "O"]

entities = ner_result

output = sa_model(text_tensor,length)
pred = torch.argmax(output, dim=1).item()
sentiment = "positive" if pred == 1 else "negative"

caption = generate_alert_image(url)

input_data = {
"text": text,
"ner": entities,
"sentiment": sentiment,
"caption": caption
}
advice = generate_investment_advice_flan(input_data)
print("Generated Advice:\n", advice)
    

/opt/conda/envs/kernel_personalizado/lib/python3.10/site-packages/torch/nn/modules/module.py:1750: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  return forward_call(*args, **kwargs)


Paste the url of the image https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F43be4e18-6efd-472e-b6eb-fa27f933015b_3600x3600.png


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/opt/conda/envs/kernel_personalizado/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Generated Advice:
 Not to invest
